<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Demo: RAG with Data Prep Kit</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

- [pdf2parquet](#pdf2parquet)
- [Chunk documents](#chunking)
- [Exact Dedup](#item3)
- [Doc_ID generation](#item4)
- [Fuzzy Dedup](#item5)
- [Language detection](#item6)
- [Doc quality](#item7)
- [Filtering](#item8)
- [Text encoder](#item9)

## Step-1: Configuration

In [1]:
## Configuration

class MyConfig:
    pass 

MY_CONFIG = MyConfig ()

In [2]:
import os 

## Input Data - configure this to the folder we want to process
MY_CONFIG.INPUT_DATA_DIR = "input_data/walmart-reports-1"

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  0.8
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 3

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

### Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

## To process a different dataset, change it here
pdf_input_folder = MY_CONFIG.INPUT_DATA_DIR 

if not os.path.exists(pdf_input_folder):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_folder = "output"

parquet_data_output = os.path.join(output_folder, "01_parquet_input")
chunk_out =  os.path.join(output_folder, "02_chunk_out")
ededup_out =  os.path.join(output_folder, "03_ededup_out")
doc_id_out =  os.path.join(output_folder, "04_doc_id_out")
fdedup_out = os.path.join(output_folder, "05_fdedup_out")
lang_out =  os.path.join(output_folder,"06_lang_out")
dq_out = os.path.join(output_folder,"07_dq_out")
filter_out = os.path.join(output_folder ,"08_filter_out")
encoder_out = os.path.join(output_folder ,"09_encoder_out")

output_folder_final = os.path.join(output_folder ,"z_final_output")

## clear output folder
shutil.rmtree(output_folder, ignore_errors=True)
shutil.os.makedirs(output_folder, exist_ok=True)

### Import Common python modules

In [4]:

import os
import sys

# Main repo root
from utils import rootdir

from data_processing_ray.runtime.ray import RayTransformLauncher
from data_processing.runtime.pure_python import PythonTransformLauncher
from data_processing.utils import ParamsUtils

<a id="pdf2parquet"></a>

## Step-2. Convert data to parquet using pdf2parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### Set Input/output Folder

In [5]:
input_folder = pdf_input_folder
output_folder =  parquet_data_output

print (f"Processing input='{input_folder}' --> output='{output_folder}'")

Processing input='input_data/walmart-reports-1' --> output='output/01_parquet_input'


### Execute 

In [6]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_code_location": ParamsUtils.convert_to_ast(code_location),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
launcher.launch()

12:27:42 INFO - Running locally
12:27:42 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': False}
12:27:42 INFO - data factory data_ is using local data access: input_folder - input_data/walmart-reports-1 output_folder - output/01_parquet_input
12:27:42 INFO - data factory data_ max_files -1, n_sample -1
12:27:42 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
12:27:42 INFO - pipeline id pipeline_id
12:27:42 INFO - code location {'github': 'github', 'commit_hash': '12345', 'path': 'path'}
12:27:42 INFO - number of workers 3 worker options {'num_cpus': 0.8, 'memory': 2147483648, 'max_restarts': -1}
12:27:42 INFO - actor creation delay 0
12:27:42 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job_id'

CPU times: user 3.82 s, sys: 666 ms, total: 4.48 s
Wall time: 6min 19s


0

<a id="chunking"></a>

##  Step-3: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### Set Input/output Folder

In [7]:
## For this stage the input is the folder containing parquet data which is output from the ingest2parquet tool

input_folder = parquet_data_output
output_folder = chunk_out

print (f"Processing input='{input_folder}' --> output='{output_folder}'")

Processing input='output/01_parquet_input' --> output='output/02_chunk_out'


### Execute 

In [8]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
launcher.launch()

12:34:00 INFO - Running locally
12:34:00 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
12:34:00 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_input output_folder - output/02_chunk_out
12:34:00 INFO - data factory data_ max_files -1, n_sample -1
12:34:00 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
12:34:00 INFO - pipeline id pipeline_id
12:34:00 INFO - code location None
12:34:00 INFO - number of workers 3 worker options {'num_cpus': 0.8, 'max_restarts': -1}
12:34:00 INFO - actor creation delay 0
12:34:00 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_chunk', 'job type'

CPU times: user 1.56 s, sys: 957 ms, total: 2.52 s
Wall time: 20 s


0

## Step-4: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### Set Input/output Folder

In [9]:
## For this stage the input is the folder containing parquet data which is output from the ingest2parquet tool

input_folder = chunk_out
output_folder = ededup_out

print (f"Processing input='{input_folder}' --> output='{output_folder}'")

Processing input='output/02_chunk_out' --> output='output/03_ededup_out'


### Execute 

In [10]:
%%time

# Import ededup transform configuration
from ededup_transform_ray import EdedupRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformConfiguration())
# launch
launcher.launch()

12:34:18 INFO - Running locally
12:34:18 INFO - exact dedup params are {'doc_column': 'contents', 'hash_cpu': 0.5, 'num_hashes': 2}
12:34:18 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_ededup_out
12:34:18 INFO - data factory data_ max_files -1, n_sample -1
12:34:18 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
12:34:18 INFO - pipeline id pipeline_id
12:34:18 INFO - code location None
12:34:18 INFO - number of workers 3 worker options {'num_cpus': 0.8, 'max_restarts': -1}
12:34:18 INFO - actor creation delay 0
12:34:18 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
2024-08-13 12:34:20,624	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(orchestrate pid=1368412) 12:34:21 INFO - orchestrator starte

CPU times: user 116 ms, sys: 139 ms, total: 255 ms
Wall time: 15.2 s


0

## Step-5:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

In [11]:
# Input for this stage is the output of exact dedeup component
# output of this component makes it possible for fdedup component to run on data.

input_folder = ededup_out
output_folder = doc_id_out
print (f"Processing input='{input_folder}' --> output='{output_folder}'")

Processing input='output/03_ededup_out' --> output='output/04_doc_id_out'


In [12]:
%%time 

from doc_id_transform_ray import DocIDRayTransformConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "hash_column",
    "doc_id_int_column": "int_id_column",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformConfiguration())
launcher.launch()

12:34:34 INFO - Running locally
12:34:34 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'hash_column', 'int_column': 'int_id_column'}
12:34:34 INFO - data factory data_ is using local data access: input_folder - output/03_ededup_out output_folder - output/04_doc_id_out
12:34:34 INFO - data factory data_ max_files -1, n_sample -1
12:34:34 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
12:34:34 INFO - pipeline id pipeline_id
12:34:34 INFO - code location None
12:34:34 INFO - number of workers 3 worker options {'num_cpus': 0.8, 'max_restarts': -1}
12:34:34 INFO - actor creation delay 0
12:34:34 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
2024-08-13 12:34:36,036	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(orchestrate pid=1370305) 12:34:

CPU times: user 122 ms, sys: 172 ms, total: 294 ms
Wall time: 15 s


0

## Step-6: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### Set Input/output Folder

In [13]:
## Input to this component is the output of doc_id generator component. 

input_folder = doc_id_out
output_folder = fdedup_out
print (f"Processing input='{input_folder}' --> output='{output_folder}'")


Processing input='output/04_doc_id_out' --> output='output/05_fdedup_out'


### Execute 

In [14]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "int_id_column",
    "fdedup_cluster_column": "hash_column",
    # infrastructure
    "fdedup_bucket_cpu": 0.5,
    "fdedup_doc_cpu": 0.5,
    "fdedup_mhash_cpu": 0.5,
    "fdedup_num_doc_actors": 2,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 2,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.8,
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())
launcher.launch()

12:34:49 INFO - Running locally
12:34:49 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'int_id_column', 'cluster_column': 'hash_column', 'bucket_cpu': 0.5, 'mhash_cpu': 0.5, 'doc_cpu': 0.5, 'num_doc_actors': 2, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 2, 'num_permutations': 64, 'threshold': 0.8, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 0.8}}
12:34:49 INFO - data factory data_ is using local data access: input_folder - output/04_doc_id_out output_folder - output/05_fdedup_out
12:34:49 INFO - data factory data_ max_files -1, n_sample -1
12:34:49 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
12:34:49 INFO - pipeline id pipeline_id
12:34:49 INFO - code location None
12:34:49 INFO - number of wo

CPU times: user 196 ms, sys: 185 ms, total: 381 ms
Wall time: 33.4 s


0

## Step-7: Language identification

This transform identifies the language of the document components.

### Set Input/output Folder

In [15]:
# input_folder = fdedup_out
# output_folder = lang_out 
# print (f"Processing input='{input_folder}' --> output='{output_folder}'")

### Execute 

In [16]:
## disabling this for now

# import os
# import sys

# from data_processing.utils import ParamsUtils
# from lang_id_transform import (
#     content_column_name_cli_param,
#     model_credential_cli_param,
#     model_kind_cli_param,
#     model_url_cli_param,
# )
# from lang_models import KIND_FASTTEXT
# from lang_id_transform_ray import LangIdentificationRayTransformConfiguration


# local_conf = {
#     "input_folder": input_folder,
#     "output_folder": output_folder,
# }
# worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
# langid_config = {
#     model_credential_cli_param: None, #"PUT YOUR OWN HUGGINGFACE CREDENTIAL",
#     model_kind_cli_param: KIND_FASTTEXT,
#     model_url_cli_param: "facebook/fasttext-language-identification",
#     # content_column_name_cli_param: "text",
# }
# params = {
#     # where to run
#     "run_locally": True,
#     # Data access. Only required parameters are specified
#     "data_local_config": ParamsUtils.convert_to_ast(local_conf),
#     # orchestrator
#     "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
#     "runtime_num_workers": 1,
#     # language selection specific parameters
#     **langid_config,
# }

# sys.argv = ParamsUtils.dict_to_req(d=params)

# # create launcher
# launcher = RayTransformLauncher(LangIdentificationRayTransformConfiguration())
# launcher.launch()


## Step-8: Document Quality

TBA

### Set Input/output Folder

In [17]:
# input_folder = lang_out
input_folder = fdedup_out
output_folder = dq_out
print (f"Processing input='{input_folder}' --> output='{output_folder}'")

Processing input='output/05_fdedup_out' --> output='output/07_dq_out'


### Execute 

In [18]:
%%time

import os
import sys
from pathlib import Path

from doc_quality_transform import (
    text_lang_cli_param,
    doc_content_column_cli_param,
    bad_word_filepath_cli_param,
)
from doc_quality_transform_ray import DocQualityRayTransformConfiguration
from data_processing.utils import ParamsUtils

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

doc_quality_basedir = os.path.join(rootdir, "transforms", "language", "doc_quality", "python")
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_creation_delay": 0,
    # doc quality configuration
    text_lang_cli_param: "en",
    doc_content_column_cli_param: "contents",
    bad_word_filepath_cli_param: os.path.join(doc_quality_basedir, "ldnoobw", "en"),
}


Path(output_folder).mkdir(parents=True, exist_ok=True)

sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocQualityRayTransformConfiguration())
# launch
launcher.launch()

12:35:22 INFO - Running locally
12:35:22 INFO - doc_quality parameters are : {'text_lang': 'en', 'doc_content_column': 'contents', 'bad_word_filepath': '/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-sujee/transforms/language/doc_quality/python/ldnoobw/en', 's3_cred': None, 'docq_data_factory': <data_processing.data_access.data_access_factory.DataAccessFactory object at 0x7ddc98071180>}
12:35:22 INFO - data factory docq_ is using local configuration without input/output path
12:35:22 INFO - data factory docq_ max_files -1, n_sample -1
12:35:22 INFO - data factory docq_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
12:35:22 INFO - data factory data_ is using local data access: input_folder - output/05_fdedup_out output_folder - output/07_dq_out
12:35:22 INFO - data factory data_ max_files -1, n_sample -1
12:35:22 INFO - data factory data_ Not using data sets, checkpointing False, max files 

CPU times: user 152 ms, sys: 175 ms, total: 326 ms
Wall time: 15.7 s


0

## Step-9: Filtering

Filter out documents that do not meet the quality threshold for each annotation. The thresholds are computed based on a distributional
analysis as well as manual inspection of samples maintaining the balance between data quality and data volume

### Set Input/output Folder

In [19]:
# input_folder = dq_out
# output_folder = filter_out
# print (f"Processing input='{input_folder}' --> output='{output_folder}'")

### Execute 

In [20]:
# ## Disabling for now

# %%time 

# import os

# from data_processing.data_access import DataAccessLocal
# from filter_transform import (
#     filter_columns_to_drop_cli_param,
#     filter_criteria_cli_param,
#     filter_logical_operator_cli_param,
# )
# from filter_transform_ray import FilterRayTransformConfiguration


# local_conf = {
#     "input_folder": input_folder,
#     "output_folder": output_folder,
# }

# # TODO
# # - decide which rules to apply for filtering


# # This is just an example criteria to filter
# filter_criteria = [
#     "total_num_lines > 10 AND total_num_lines < 90",
#     "lang_selected = 1",
# ]
# filter_logical_operator = "AND"
# filter_columns_to_drop = ["lang_selected", "hash_column"]

# filter_params = {
#     filter_criteria_cli_param: filter_criteria,
#     filter_columns_to_drop_cli_param: filter_columns_to_drop,
#     filter_logical_operator_cli_param: filter_logical_operator,
# }

# worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
# launcher_params = {
#     # where to run
#     "run_locally": True,
#     # Data access. Only required parameters are specified
#     "data_local_config": ParamsUtils.convert_to_ast(local_conf),
#     # orchestrator
#     "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
#     "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
# }


# sys.argv = ParamsUtils.dict_to_req(launcher_params | filter_params)
# # Create the longer to launch with the blocklist transform.
# launcher = RayTransformLauncher(FilterRayTransformConfiguration())
# # Launch the ray actor(s) to process the input
# launcher.launch()

## Step-10:   Text encoding

Encode text for the vector storage.

In [21]:
# input_folder = filter_out
input_folder = dq_out
output_folder = encoder_out
print (f"Processing input='{input_folder}' --> output='{output_folder}'")

Processing input='output/07_dq_out' --> output='output/09_encoder_out'


In [22]:
%%time 

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input
launcher.launch()


12:35:38 INFO - Running locally
12:35:38 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
12:35:38 INFO - data factory data_ is using local data access: input_folder - output/07_dq_out output_folder - output/09_encoder_out
12:35:38 INFO - data factory data_ max_files -1, n_sample -1
12:35:38 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
12:35:38 INFO - pipeline id pipeline_id
12:35:38 INFO - code location None
12:35:38 INFO - number of workers 3 worker options {'num_cpus': 0.8, 'max_restarts': -1}
12:35:38 INFO - actor creation delay 0
12:35:38 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
2024-08-13 12:35:40,633	INFO worker.py:1744 -- Started a local Ray instance. View the d

CPU times: user 398 ms, sys: 248 ms, total: 645 ms
Wall time: 56.7 s


0

## Step-11: Copy output to final output dir

In [23]:
import shutil

print (f"Copying output from '{encoder_out}' --> '{output_folder_final}'")

shutil.rmtree(output_folder_final, ignore_errors=True)
shutil.copytree(src=encoder_out, dst=output_folder_final)

Copying output from 'output/09_encoder_out' --> 'output/z_final_output'


'output/z_final_output'